# 批量预测

在进行模型评估或其他任务时，通常需要对大量数据进行预测。然而，模型推理过程往往耗时较长，通过循环串行执行会增加整体时间成本，而并行执行则需要额外的开发工作。

SDK 提供了多种解决方案来应对这一场景，其中包括：

- [本地并行推理](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/batch_prediction.ipynb)：利用 SDK 内置的批量推理功能，在本地通过并行调用模型接口实现高效的批量预测。
- [数据集评估](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/dataset/batch_inference_using_dataset.ipynb)：利用 SDK 的 Dataset 模块，调用平台提供的数据集评估功能，以便快速而有效地完成任务。
- [离线批量推理](https://github.com/baidubce/bce-qianfan-sdk/blob/main/cookbook/offline_batch_inference.ipynb)：对于时间要求不那么严格的场景，可以考虑利用平台提供的离线批量预测能力，以降低实时推理的负载压力。

本文将介绍第一种解决方案，即本地并行推理。

注意：本文的方法需要 SDK 版本 >= 0.1.4

In [1]:
import qianfan
import os

# 这里请根据 SDK 文档获取自己的 access key 和 secret key
os.environ["QIANFAN_ACCESS_KEY"] = "your access key"
os.environ["QIANFAN_SECRET_KEY"] = "your secret key"
# 由于并行会带来较大的并发量，容易触发 QPS 限制导致请求失败
# 因此建议这里为 SDK 设置一个合理的 QPS 限制，SDK 会根据该数值进行流控
os.environ["QIANFAN_QPS_LIMIT"] = "3"

## 准备数据

需要准备的数据与需要使用的模型类型相关，例如 `Completion` 接受的参数 `prompt` 类型是 `str`，那么批量预测时需要使用的就是 `List[str]`。

这里以 CMMLU 数据集为例，使用 `Completion` 模型进行批量预测。

In [2]:
# 这里使用 HuggingFace 上的数据集，如果已经有数据集可以跳过这步
!pip install datasets

In [3]:
import datasets
dataset = datasets.load_dataset("haonan-li/cmmlu", 'chinese_literature')

In [4]:
# 这里仅取前 10 个样本进行测试
# data 只需要是 List[str] 类型即可
data = dataset['test']['Question'][:10]
for i in data:
    print(i)

《日出》的结构采用的是
以眉间尺为父报仇作为中心线索的小说是
“他只是遗憾／他的祖先没有像他一样想过／不然，见到大海的该是他了”。以此作为结尾的诗歌是
下列含有小说《伤逝》的鲁迅作品集是
汪静之属于
杂文《春末闲谈》中，细腰蜂的故事所要表达的是
臧克家的第一部诗集是
下列小说中运用了“戏剧穿插法”的是
台静农的《记波外翁》中波外翁的性格是
下列哪一项不属于巴金的短篇小说集


## 批量预测

我们可以将需要预测的所有数据放在一个 List 中，然后调用 `batch_do` 方法，SDK 会在后台启动数个线程并行处理，线程数量由 `worker` 参数指定。

In [5]:
r = qianfan.Completion().batch_do(data, worker_num=5)

[INFO] [02-21 16:12:51] openapi_requestor.py:244 [t:13181227008]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:51] openapi_requestor.py:244 [t:13147648000]: requesting llm api endpoint: /chat/eb-instant


返回的结果 `r` 是一个 future 对象，可以调用 `r.result()` 等待所有数据预测完成，并返回结果。

或者也可以直接遍历 r，逐个获取结果，在遇到未完成的任务时再等待。

In [6]:
results = r.results()
# 这里为了展示结果，仅取了前 3 个结果
# 实际使用时可以去掉该限制
for i, (input, output) in enumerate(zip(data[:3], results)):
    print(f"问题{i}：")
    print(input)
    print(output['result'])
    print("=================")

# 或者
for i, (input, output) in enumerate(zip(data, r)):
    print(f"问题{i}：")
    print(input)
    print(output.result()['result'])
    print("=================")

[INFO] [02-21 16:12:51] openapi_requestor.py:244 [t:13198016512]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:51] openapi_requestor.py:244 [t:13231595520]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:51] openapi_requestor.py:244 [t:13214806016]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:51] utils.py:126 [t:13181227008]: no event loop in thread `ThreadPoolExecutor-2_1`, async feature won't be available. Please make sure the object is initialized in the thread with event loop.
[INFO] [02-21 16:12:51] utils.py:126 [t:13147648000]: no event loop in thread `ThreadPoolExecutor-2_0`, async feature won't be available. Please make sure the object is initialized in the thread with event loop.
[INFO] [02-21 16:12:51] utils.py:126 [t:13198016512]: no event loop in thread `ThreadPoolExecutor-2_2`, async feature won't be available. Please make sure the object is initialized in the thread with event loop.
[INFO] [02-21 16:12:51] util

问题0：
《日出》的结构采用的是
《日出》是曹禺的作品，它的结构采用的是戏剧结构。这部剧作通过一个普通小旅馆中的一群人物的群戏，展示了现实社会中压迫者的凶残、被压迫者的善良，并以其强大的社会纵深感揭示了社会矛盾的复杂性。

曹禺的戏剧结构注重情节的紧凑和人物关系的构建，通过精心设计的情节和人物关系，将各种矛盾冲突推向高潮，从而达到揭示社会矛盾和人性冲突的目的。同时，曹禺也善于运用象征、暗示、隐喻等手法，增强戏剧的视觉化和艺术性。

《日出》的结构遵循了戏剧的结构原则，通过一系列的事件和人物活动，逐步展现出复杂的社会矛盾和人性的挣扎。因此，可以认为《日出》的结构采用的是戏剧结构。
问题1：
以眉间尺为父报仇作为中心线索的小说是
以眉间尺为父报仇作为中心线索的小说有：

1. 《眉间尺》是孙见喜所著的历史小说，讲述春秋时期吴越争霸时期，忠臣伍子胥报仇雪恨的故事。
2. 《血仇》是刘流所著的小说，讲述了眉间尺为父报仇的故事。

此外，还有《中国剑侠传奇：眉间尺》等。如果您需要更多信息，可以到相关网站上查询。
问题2：
“他只是遗憾／他的祖先没有像他一样想过／不然，见到大海的该是他了”。以此作为结尾的诗歌是
根据您提供的诗歌，我理解这首诗可能反映了一种失落或者对于自己生活和身份的不满情绪。对于海洋的渴望可能是渴望更大的冒险，或者探索新的可能性。这样的情绪可以被描述为对过去的某种“遗憾”，也许是过去的人或事物没有带来现在的“他”渴望的视野和可能性。至于最后一句，“不然，见到大海的该是他了”，这句暗示了某种改变或进步的可能性，也许是一种对未来的期待或希望。

这首诗的结尾部分，并没有明确表达出“他”的具体身份和背景，所以我无法确定这首诗的具体主题或类型。但是，从诗歌的情感和语境来看，它可能是一首反映个人内心世界，对生活和身份不满的作品。

不过这只是一个初步的分析，可能并不能完全准确解读这首诗的所有内涵。每个诗人或作品都有其独特的特点和背景，可能需要对特定的环境和语境进行更深入的研究和理解。希望这个答案能帮到您。
问题0：
《日出》的结构采用的是
《日出》是曹禺的作品，它的结构采用的是戏剧结构。这部剧作通过一个普通小旅馆中的一群人物的群戏，展示了现实社会中压迫者的凶残、被压迫者的善良，并以其强大的社会纵深感揭示了社会矛盾的复杂性。

曹禺的戏剧结构注重情节的紧凑和人物关系的构建

`r.results()` 会同时返回所有结果，当数据量较大时可能会占用大量内存，可以调用 `r.wait()` 仅等待而不返回结果。

此外可以通过 `r.finished_count()` 获取已完成的任务数量，`r.task_count()` 获取任务总数量。

In [7]:
# 获取完成的比例
print("{}/{}".format(r.finished_count(), r.task_count()))

10/10


对于其他类型模型，数据 `List` 中的元素类型需要进行改变，例如 `ChatCompletion` 接受的参数是 Messages List，那么就要进行转换。

In [8]:
chat_data = [[{
    "role": "user",
    "content": question
}] for question in data]

In [9]:
r = qianfan.ChatCompletion().batch_do(chat_data, worker_num=5)
r.wait()
print(r.results()[0]['result'])

[INFO] [02-21 16:12:57] openapi_requestor.py:244 [t:13147648000]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:57] openapi_requestor.py:244 [t:13181227008]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:57] openapi_requestor.py:244 [t:13214806016]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:57] openapi_requestor.py:244 [t:13198016512]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:57] openapi_requestor.py:244 [t:13231595520]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:58] openapi_requestor.py:244 [t:13214806016]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:12:59] openapi_requestor.py:244 [t:13231595520]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:00] openapi_requestor.py:244 [t:13181227008]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:00] openapi_requestor.py:244 [t:13198016512]: requesting llm api endpoint: /chat/eb-

《日出》的结构采用的是戏剧中最常见的一种“三一律”，即整个戏剧必须在3小时内演完，并集中在同一场景，通过一件事件来展开情节。该剧分为3幕，每一幕又可以划分为3个小节，总共9个场次。这种结构形式使得《日出》在有限的场景和时间内，紧凑地展现出人物的性格和命运，同时也为观众提供了强烈的节奏感。

此外，《日出》的结构还采用了“间离效果”的手法，即通过运用象征、荒诞和幽默等元素，使观众在观看的过程中产生出一种疏离感，从而引发对现实世界的反思。这种手法打破了传统戏剧的叙事方式，使得剧情更加引人深思。

因此，《日出》的结构采用了“三一律”的戏剧结构形式，紧凑地展现了人物的性格和命运，同时也采用了“间离效果”的手法，使得剧情更加引人深思。


如果推理过程发生错误，会在调用 `result()` 时抛出异常，请注意进行异常处理

In [10]:
err_data=["你好", [{"role":"user", "content":"你好"}]]

r = qianfan.ChatCompletion().batch_do(err_data)

for i, res in enumerate(r):
    try:
        print("数据{}结果：{}".format(i, res.result()['result']))
    except Exception as e:
        print(f"数据{i}发生异常：{e}")

[INFO] [02-21 16:13:04] openapi_requestor.py:244 [t:13147648000]: requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:244 [t:13181227008]: requesting llm api endpoint: /chat/eb-instant
[ERROR] [02-21 16:13:04] openapi_requestor.py:219 [t:13147648000]: api request req_id: as-n30m197kxm failed with error code: 336002, err msg: Invalid JSON, please check https://cloud.baidu.com/doc/WENXINWORKSHOP/s/tlmyncueh


数据0发生异常：api return error, req_id: as-n30m197kxm code: 336002, msg: Invalid JSON
数据1结果：你好，有什么我可以帮助你的吗？


[INFO] [02-21 16:13:04] base.py:89 [t:13181227008]: All tasks finished, exeutor will be shutdown


## 异步调用

In [11]:
results = await qianfan.Completion().abatch_do(data, worker_num=5)
# 返回值为一个 List，与输入列表中的元素一一对应
# 正常情况下与 `ado` 返回类型一致，但如果发生异常则会是一个 Exception 对象
for prompt, result in zip(data[:3], results):
    if not isinstance(result, Exception):
        print(prompt)
        print(result['result'])
        print("==============")

[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]:

同样的异步场景下，如果发生了错误需要进行异常处理，此时 results 中某个数据发生错误，那么对应的对象会是 Excepiton 类型，可以通过 `isinstance` 进行识别。

In [12]:
err_data=["你好", [{"role":"user", "content":"你好"}]]

results = await qianfan.ChatCompletion().abatch_do(err_data)

[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant
[INFO] [02-21 16:13:04] openapi_requestor.py:275 [t:8644182656]: async requesting llm api endpoint: /chat/eb-instant


In [13]:
for i, result in enumerate(results):
    if not isinstance(result, Exception):
        print("数据{}结果：{}".format(i, result['result']))
    else:
        print("数据{}发生错误：{}".format(i, result))

数据0发生错误：no event loop found in current thread, please make sure the event loop is available when the object is initialized
数据1发生错误：no event loop found in current thread, please make sure the event loop is available when the object is initialized
